# 第10章 性能評価

## 10.2 評価指標を用いた自動評価

### 10.2.4 多肢選択式質問応答タスクによる自動評価

#### 環境の準備

In [ ]:
!pip install transformers[torch,sentencepiece] bitsandbytes datasets

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from transformers.trainer_utils import set_seed

# 乱数シードを42に固定する
set_seed(42)

#### データセットの準備

In [ ]:
from datasets import load_dataset

# データセットを読み込む
train_dataset = load_dataset(
    "llm-book/JGLUE", name="JCommonsenseQA", split="train"
)
val_dataset = load_dataset(
    "llm-book/JGLUE", name="JCommonsenseQA", split="validation"
)
print(val_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for llm-book/JGLUE contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/llm-book/JGLUE.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['q_id', 'question', 'choice0', 'choice1', 'choice2', 'choice3', 'choice4', 'label'],
    num_rows: 1119
})


#### データの前処理

In [ ]:
from pprint import pprint

def convert_data_format(data: dict[str, str]) -> dict[str, str]:
    """選択肢の中から質問に数字で回答する形式にデータを変換する"""
    data["input"] = (
        f"質問：{data['question']}\n"
        f"選択肢：0.{data['choice0']},1.{data['choice1']},"
        f"2.{data['choice2']},3.{data['choice3']},"
        f"4.{data['choice4']}"
    )
    data["output"] = data['label']
    return data

# 訓練セットをシャッフルする
train_dataset = train_dataset.shuffle()
# 訓練セットの前処理をする
train_dataset = train_dataset.map(convert_data_format)
# 四つのfew-shot事例を取得する
few_shots = list(train_dataset)[:4]
# 検証セットの前処理をする
val_dataset = val_dataset.map(convert_data_format)
pprint(list(val_dataset)[0])

Map:   0%|          | 0/8939 [00:00<?, ? examples/s]

Map:   0%|          | 0/1119 [00:00<?, ? examples/s]

{'choice0': '掲示板',
 'choice1': 'パソコン',
 'choice2': 'マザーボード',
 'choice3': 'ハードディスク',
 'choice4': 'まな板',
 'input': '質問：電子機器で使用される最も主要な電子回路基板の事をなんと言う？\n'
          '選択肢：0.掲示板,1.パソコン,2.マザーボード,3.ハードディスク,4.まな板',
 'label': 2,
 'output': '2',
 'q_id': 8939,
 'question': '電子機器で使用される最も主要な電子回路基板の事をなんと言う？'}


#### プロンプトテンプレートの作成

In [ ]:
def create_prompt_template(
    instruction: str, few_shots: list[dict[str, str]] | None = None
) -> str:
    """プロンプトテンプレートを作成する"""
    prompt_template = (
        "以下は、タスクを説明する指示と、"
        "文脈のある入力の組み合わせです。"
        "要求を適切に満たす応答を書きなさい。\n\n"
    )
    prompt_template += f"### 指示:\n{instruction}\n\n"
    if few_shots is not None:
        for few_shot in few_shots:
            prompt_template += f"### 入力:\n{few_shot['input']}\n\n"
            prompt_template += f"### 応答:\n{few_shot['output']}\n\n"
    prompt_template += "### 入力:\n{input}\n\n"
    prompt_template += "### 応答:\n"
    return prompt_template

# 指示文を指定してプロンプトテンプレートを作成する
instruction = "質問と回答の選択肢を入力として受け取り、選択肢から回答を選択してください。なお、回答は選択肢の番号（例：0）でするものとします。 回答となる数値をint型で返し、他には何も含めないことを厳守してください。"
prompt_template = create_prompt_template(instruction, few_shots)
print(prompt_template)

以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示:
質問と回答の選択肢を入力として受け取り、選択肢から回答を選択してください。なお、回答は選択肢の番号（例：0）でするものとします。 回答となる数値をint型で返し、他には何も含めないことを厳守してください。

### 入力:
質問：4輪でハンドルで操作し、ガソリンや電気で動く乗り物は？
選択肢：0.自動車,1.自転車,2.イヤホン,3.飛行機,4.ライブ

### 応答:
0

### 入力:
質問：夏になったら着たくなるものは？
選択肢：0.誘い水,1.水着,2.化粧水,3.水すまし,4.水筒

### 応答:
1

### 入力:
質問：声や楽器を使った芸術は？
選択肢：0.猫,1.音楽,2.展覧会,3.歌,4.スズメ

### 応答:
1

### 入力:
質問：売る物のことを何と言うか？
選択肢：0.自転車,1.鍋,2.車,3.飲み物,4.商品

### 応答:
4

### 入力:
{input}

### 応答:



#### パイプラインの作成

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)

model_name = "tokyotech-llm/Swallow-7b-hf"
# AutoTokenizerでトークナイザを読み込む
tokenizer = AutoTokenizer.from_pretrained(model_name)
# モデルを量子化して読み込むためのパラメータを指定する
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
# 生成を行うモデルであるAutoModelForCausalLMを使ってモデルを読み込む
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
    use_cache=False,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/773 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/914k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/457 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.68G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/203 [00:00<?, ?B/s]

In [ ]:
# テキスト生成用のパラメータを指定する
generation_config = {
    "max_new_tokens": 1, # 生成する最大トークン数
    "top_p": 1.0, # top-pサンプリング
    "repetition_penalty": 1.0, # 繰り返しペナルティ
}
# pipelineを作成する
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    **generation_config
)

#### 質問の回答を生成

In [ ]:
from datasets import Dataset
from tqdm import tqdm
from transformers import TextGenerationPipeline

def generate_answers(
    text_generation_pipeline: TextGenerationPipeline,
    dataset: Dataset,
    prompt_template: str,
) -> list[dict[str, str]]:
    """プロンプトを使って質問の回答を生成する"""
    results = []
    for data in tqdm(dataset):
        # プロンプトテンプレートの{input}を質問テキストに置換する
        prompt = prompt_template.format(input=data["input"])
        # 質問の回答を生成する
        output = text_generation_pipeline(prompt)
        # プロンプト部分を削除して予測部分のみにする
        generated_text = output[0]["generated_text"].replace(
            prompt, ""
        )
        # 複数行出力された場合に、最初の行だけを抽出して回答部分のみにする
        pred_label = generated_text.split("\n")[0].strip()
        results.append(
            {
                "input": data["input"],
                "true_label": data["output"],
                "pred_label": pred_label,
            }
        )
    return results

# 検証セットに対して質問の回答を生成する
results1 = generate_answers(
    text_generation_pipeline, val_dataset, prompt_template
)
pprint(results1[:3])

100%|██████████| 1119/1119 [01:43<00:00, 10.84it/s]

[{'input': '質問：電子機器で使用される最も主要な電子回路基板の事をなんと言う？\n'
           '選択肢：0.掲示板,1.パソコン,2.マザーボード,3.ハードディスク,4.まな板',
  'pred_label': '2',
  'true_label': '2'},
 {'input': '質問：田んぼが広がる風景を何という？\n選択肢：0.畑,1.海,2.田園,3.地方,4.牧場',
  'pred_label': '2',
  'true_label': '2'},
 {'input': '質問：しゃがんだりする様を何という？\n選択肢：0.腰を下す,1.座る,2.仮眠を取る,3.寝る,4.起きる',
  'pred_label': '3',
  'true_label': '0'}]


In [ ]:
from google.colab import drive

# Googleドライブを"drive"ディレクトリ以下にマウントする
drive.mount("drive")

Mounted at drive


In [ ]:
import json
from pathlib import Path

def write_jsonl(path: str, items: list[dict]) -> None:
    """JSON Lines形式で出力する"""
    # 保存先のフォルダが存在しない場合は作成する
    Path(path).parent.mkdir(parents=True, exist_ok=True)

    # ファイルに書き込む
    with open(path, "w") as f:
        for item in items:
            print(json.dumps(item, ensure_ascii=False), file=f)

# 出力結果をresult1.jsonlというファイルに書き込む
output_path = "./drive/MyDrive/llm_book/eval/jcommonsenseqa/results1.jsonl"
write_jsonl(output_path, results1)

#### 完全一致率の算出

In [ ]:
def calc_exact_match_ratio(
    trues: list[str], preds: list[str]
) -> float:
    """完全一致率を算出する"""
    # どちらかの事例がなければ0
    if len(trues) == 0 or len(preds) == 0:
        return 0
    # 正解テキストと予測テキストが一致していれば1、そうでなければ0
    num_exact_match = sum(
        1 if t == p else 0 for t, p in zip(trues, preds)
    )
    return num_exact_match / len(trues)

# 完全一致率を算出する
true_labels1 = [r["true_label"] for r in results1]
pred_labels1 = [r["pred_label"] for r in results1]
score = calc_exact_match_ratio(true_labels1, pred_labels1)
print("完全一致率: ", score)

完全一致率:  0.4334226988382484


#### 文字列回答形式の評価結果

In [ ]:
def convert_data_format2(data: dict[str, str]) -> dict[str, str]:
    """選択肢の中から質問に文字列で回答する形式にデータを変換する"""
    data["input"] = (
        f"質問：{data['question']}\n"
        f"選択肢：{data['choice0']},{data['choice1']},"
        f"{data['choice2']},{data['choice3']},"
        f"{data['choice4']}"
    )
    choice = f"choice{data['label']}"
    data["output"] = data[choice]
    return data

# 訓練セットの前処理をする
train_dataset = train_dataset.map(convert_data_format2)
# 四つのfew-shot事例を取得する
few_shots2 = list(train_dataset)[:4]
# 開発セットの前処理をする
val_dataset = val_dataset.map(convert_data_format2)
print(list(val_dataset)[0])

Map:   0%|          | 0/8939 [00:00<?, ? examples/s]

Map:   0%|          | 0/1119 [00:00<?, ? examples/s]

{'q_id': 8939, 'question': '電子機器で使用される最も主要な電子回路基板の事をなんと言う？', 'choice0': '掲示板', 'choice1': 'パソコン', 'choice2': 'マザーボード', 'choice3': 'ハードディスク', 'choice4': 'まな板', 'label': 2, 'input': '質問：電子機器で使用される最も主要な電子回路基板の事をなんと言う？\n選択肢：掲示板,パソコン,マザーボード,ハードディスク,まな板', 'output': 'マザーボード'}


In [ ]:
# プロンプトテンプレートを作成する
instruction2 = "質問と回答の選択肢を入力として受け取り、選択肢から回答を選択してください。なお、回答以外には何も含めないことを厳守してください。"
prompt_template2 = create_prompt_template(instruction2, few_shots2)
print(prompt_template2)

以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示:
質問と回答の選択肢を入力として受け取り、選択肢から回答を選択してください。なお、回答以外には何も含めないことを厳守してください。

### 入力:
質問：4輪でハンドルで操作し、ガソリンや電気で動く乗り物は？
選択肢：自動車,自転車,イヤホン,飛行機,ライブ

### 応答:
自動車

### 入力:
質問：夏になったら着たくなるものは？
選択肢：誘い水,水着,化粧水,水すまし,水筒

### 応答:
水着

### 入力:
質問：声や楽器を使った芸術は？
選択肢：猫,音楽,展覧会,歌,スズメ

### 応答:
音楽

### 入力:
質問：売る物のことを何と言うか？
選択肢：自転車,鍋,車,飲み物,商品

### 応答:
商品

### 入力:
{input}

### 応答:



In [ ]:
# テキスト生成用のパラメータを指定する
generation_config2 = {
    "max_new_tokens": 10,
    "top_p": 1.0,
    "repetition_penalty": 1.0,
}
# pipelineを作成する
text_generation_pipeline2 = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    **generation_config2
)

In [ ]:
# 検証セットに対して質問の回答を生成する
results2 = generate_answers(
    text_generation_pipeline2, val_dataset, prompt_template2
)
pprint(results2[:3])
output_path = "./drive/MyDrive/llm_book/eval/jcommonsenseqa/results2.jsonl"
write_jsonl(output_path, results2)

100%|██████████| 1119/1119 [12:03<00:00,  1.55it/s]


[{'input': '質問：電子機器で使用される最も主要な電子回路基板の事をなんと言う？\n選択肢：掲示板,パソコン,マザーボード,ハードディスク,まな板',
  'pred_label': 'マザーボード',
  'true_label': 'マザーボード'},
 {'input': '質問：田んぼが広がる風景を何という？\n選択肢：畑,海,田園,地方,牧場',
  'pred_label': '田園',
  'true_label': '田園'},
 {'input': '質問：しゃがんだりする様を何という？\n選択肢：腰を下す,座る,仮眠を取る,寝る,起きる',
  'pred_label': '座る',
  'true_label': '腰を下す'}]


In [ ]:
# 完全一致率を算出する
true_labels2 = [r["true_label"] for r in results2]
pred_labels2 = [r["pred_label"] for r in results2]
score2 = calc_exact_match_ratio(true_labels2, pred_labels2)
print("完全一致率:", score2)

完全一致率: 0.7479892761394102
